In [1]:
from functools import reduce
import re
import pandas as pd
import numpy as np
from scipy.spatial.distance import mahalanobis
from scipy.optimize import linear_sum_assignment
import warnings

warnings.filterwarnings('ignore')

ЗДЕСЬ И СОБЫТИЯ ЯНДЕКСА И РОССТАТ

In [2]:
events1 = pd.read_csv('события.csv')
events2 = pd.read_csv('росстат_общий_new.csv')


df = events2.merge(
    events1[['Event Text', 'region_name', 'municipality', 'year']],
    on=['municipality', 'region_name', 'year'],
    how='left'
)


df.to_csv('общие_данные_2012t.csv', index=False)
df

,municipality,oktmo,Процент (рус),region_name,year,Жилье (1000 м2),Дороги (%),Канализация (1 м),Водопровод (1 метр),active_percent,city_percent,value,Event Text
0,NaN,28602100,0.000000,Тверская,2010.0,1.9,0.0,0.0,0.0,0.000000,0.0,0.0,NaN
1,NaN,28602100,0.000000,Тверская,2012.0,2.2,0.0,0.0,0.0,0.000000,0.0,0.0,NaN
2,NaN,28602100,0.000000,Тверская,2014.0,2.2,0.0,0.0,0.0,0.000000,0.0,0.0,NaN
3,NaN,28602100,0.000000,Тверская,2015.0,0.0,0.0,100.0,300.0,0.000000,0.0,0.0,NaN
4,NaN,28602400,0.000000,Тверская,2010.0,14.3,0.0,0.0,0.0,0.000000,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
256415,имени Полины Осипенко,8637000,91.832956,Хабаровский,2022.0,0.0,79.6,0.0,0.0,54.528488,0.0,0.0,NaN
256416,имени Полины Осипенко,8637000,91.832956,Хабаровский,2023.0,0.0,80.3,0.0,0.0,54.432127,0.0,0.0,NaN
256417,имени Полины Осипенко,8637406,93.960350,Хабаровский,2007.0,0.4,0.0,0.0,0.0,0.000000,0.0,0.0,NaN
256418,имени Полины Осипенко,8637406,93.960350,Хабаровский,2008.0,0.4,0.0,0.0,0.0,0.000000,0.0,0.0,NaN


ОБЪЕДИНЕНИЕ ПРОТЕСТОВ С ДАННЫМИ РОССТАТА 

1. ВООБЩЕ 
2. ТОЛЬКО ЗА 2012

In [3]:
data = pd.read_csv('общие_данные_2012t.csv')

protest_events = ["Demonstrate or rally", "Criticize or denounce", "Defy norms, law"]


protests = data[data['Event Text'].isin(protest_events)].copy()
protests_0910 = protests[protests['year'].isin([2011, 2012, 2013])].copy()

# Для каждого oktmo записываем минимальный год протеста в 2009–2010
protests_0910['fprotets_year'] = protests_0910.groupby('oktmo')['year'].transform('min')

# Оставляем по одной строке на oktmo (колонка fprotets_year уже заполнена правильно)
protests1 = protests_0910.drop_duplicates(subset=['oktmo']).reset_index(drop=True)



protests.to_csv('протесты_2012t.csv', index=False)
protests1.to_csv('протесты2012_2012t.csv', index=False)
protests1

,municipality,oktmo,Процент (рус),region_name,year,Жилье (1000 м2),Дороги (%),Канализация (1 м),Водопровод (1 метр),active_percent,city_percent,value,Event Text,fprotets_year
0,Абазинский,91601000,0.000000,Карачаево-Черкесская,2013.0,0.0,40.14,0.0,0.0,45.329956,0.000000,1.062859e+06,Criticize or denounce,2013.0
1,Агрызский,92601000,25.197968,Татарстан,2012.0,29.0,84.09,0.0,0.0,37.185867,54.735351,4.810534e+06,Criticize or denounce,2012.0
2,Алнашский,94602000,12.805933,Удмуртская,2012.0,12.3,1.57,182.0,1781.0,41.614192,0.000000,1.887755e+06,Demonstrate or rally,2012.0
3,Альшеевский,80602000,24.021840,Башкортостан,2012.0,0.0,90.00,100.0,1500.0,43.207559,0.000000,4.494512e+06,Demonstrate or rally,2012.0
4,Андреапольский,28602000,97.665070,Тверская,2011.0,20.4,0.00,0.0,400.0,38.717237,60.226412,1.184288e+06,Criticize or denounce,2011.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143,Южно-Сахалинск,64701000,77.693897,Сахалинская,2011.0,123.6,96.48,200.0,700.0,25.110739,96.155110,8.905167e+07,Criticize or denounce,2011.0
144,Якутск,98701000,36.968829,Саха,2011.0,333.7,10.00,200.0,4500.0,58.011502,94.271863,9.172549e+07,Criticize or denounce,2011.0
145,Янаульский,80659000,15.041336,Башкортостан,2012.0,20.0,14.20,0.0,5800.0,43.021814,56.961782,5.047203e+06,Criticize or denounce,2012.0
146,Ярославль,78701000,0.000000,Ярославская,2012.0,283.1,25.42,3200.0,12560.0,39.430050,100.000000,1.556020e+08,Demonstrate or rally,2012.0


ЕДИНИЦЫ БЕЗ ПРОТЕСТОВ СРЕДИ ВСЕХ СОБЫТИЙ ЗА ВСЕ ВРЕМЯ

In [4]:
lol = pd.read_csv("протесты_2012t.csv") 
sobytia = pd.read_csv("общие_данные_2012t.csv")


noprotestALL = sobytia[~sobytia['municipality'].isin(lol['municipality'])].reset_index(drop=True)
noprotestALL.to_csv('единицы_без_протестов_2012t.csv', index=False)
noprotestALL

,municipality,oktmo,Процент (рус),region_name,year,Жилье (1000 м2),Дороги (%),Канализация (1 м),Водопровод (1 метр),active_percent,city_percent,value,Event Text
0,NaN,28602100,0.000000,Тверская,2010.0,1.9,0.0,0.0,0.0,0.000000,0.0,0.0,NaN
1,NaN,28602100,0.000000,Тверская,2012.0,2.2,0.0,0.0,0.0,0.000000,0.0,0.0,NaN
2,NaN,28602100,0.000000,Тверская,2014.0,2.2,0.0,0.0,0.0,0.000000,0.0,0.0,NaN
3,NaN,28602100,0.000000,Тверская,2015.0,0.0,0.0,100.0,300.0,0.000000,0.0,0.0,NaN
4,NaN,28602400,0.000000,Тверская,2010.0,14.3,0.0,0.0,0.0,0.000000,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
175022,имени Полины Осипенко,8637000,91.832956,Хабаровский,2022.0,0.0,79.6,0.0,0.0,54.528488,0.0,0.0,NaN
175023,имени Полины Осипенко,8637000,91.832956,Хабаровский,2023.0,0.0,80.3,0.0,0.0,54.432127,0.0,0.0,NaN
175024,имени Полины Осипенко,8637406,93.960350,Хабаровский,2007.0,0.4,0.0,0.0,0.0,0.000000,0.0,0.0,NaN
175025,имени Полины Осипенко,8637406,93.960350,Хабаровский,2008.0,0.4,0.0,0.0,0.0,0.000000,0.0,0.0,NaN


ПОИСК КОНТРОЛЬНОЙ ГРУППЫ (НЕПРОТЕСТНЫЕ ЕДИНИЦЫ ЗА 2011 ИЗ ТЕХ ЧТО ПРОТЕСТНЫЕ В 2012)

In [5]:
lol = pd.read_csv("протесты2012_2012t.csv") 
sobytia = pd.read_csv("общие_данные_2012t.csv")


df_2011 = sobytia[sobytia['year'] == 2010]


filtered_df = df_2011[df_2011['oktmo'].isin(lol['oktmo'])]

pattern = "Demonstrate or rally|Criticize or denounce|Defy norms, law"
mask = filtered_df['Event Text'].str.contains(pattern, case=False, na=False)
municipalities_with_tag = filtered_df[mask]['oktmo'].unique()


final_df = filtered_df[~filtered_df['oktmo'].isin(municipalities_with_tag)].reset_index(drop=True)

final_df = final_df.drop_duplicates(subset='oktmo').reset_index(drop=True)
final_df.to_csv('без_протестов2011_2012t.csv', index=False)
final_df



,municipality,oktmo,Процент (рус),region_name,year,Жилье (1000 м2),Дороги (%),Канализация (1 м),Водопровод (1 метр),active_percent,city_percent,value,Event Text
0,Абазинский,91601000,0.000000,Карачаево-Черкесская,2010.0,0.00,0.00,0.0,3800.0,0.0,0.0,0.0,NaN
1,Алнашский,94602000,12.805933,Удмуртская,2010.0,22.40,0.00,0.0,660.0,0.0,0.0,0.0,Investigate
2,Апатиты,47705000,93.697785,Мурманская,2010.0,1.40,0.00,300.0,500.0,0.0,0.0,0.0,NaN
3,Архангельский,80603000,40.736062,Башкортостан,2010.0,6.20,0.00,0.0,0.0,0.0,0.0,0.0,NaN
4,Асекеевский,53607000,40.317393,Оренбургская,2010.0,7.74,0.00,0.0,3500.0,0.0,0.0,0.0,Demand
...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,Элиста,85701000,24.905577,Калмыкия,2010.0,51.30,0.00,0.0,2963.0,0.0,0.0,0.0,Reject
90,Южно-Сахалинск,64701000,77.693897,Сахалинская,2010.0,171.00,1.03,0.0,4700.0,0.0,0.0,0.0,Consult
91,Якутск,98701000,36.968829,Саха,2010.0,311.76,0.00,1800.0,1300.0,0.0,0.0,0.0,"Arrest, detain, or charge with legal action"
92,Янаульский,80659000,15.041336,Башкортостан,2010.0,10.70,0.00,200.0,5000.0,0.0,0.0,0.0,Make statement


In [6]:
def check_years_completeness(oktmo, df_events_full, required_years):
    municipality_data = df_events_full[df_events_full['oktmo'] == oktmo]
    existing_years = set(municipality_data['year'].unique())
    return existing_years.issuperset(required_years)



df_protests = pd.read_csv("протесты2012_2012t.csv")
df_protests = df_protests.drop_duplicates(subset=['region_name', 'municipality'])
df_events = pd.read_csv("единицы_без_протестов_2012t.csv")
df_events_full = pd.read_csv("общие_данные_2012t.csv")

required_years = {2010.0, 2014.0, 2015.0, 2016.0}


df_events = df_events[df_events['year'].isin([2011, 2012, 2013])].reset_index(drop=True)

features = [
    "Жилье (1000 м2)", "Дороги (%)", "Канализация (1 м)",
    "Водопровод (1 метр)"
]

df_protests = df_protests[['region_name', 'municipality', 'oktmo', 'fprotets_year'] + features].reset_index(drop=True)
df_events = df_events[['region_name', 'municipality', 'oktmo'] + features].reset_index(drop=True)

print("Размер df_protests:", len(df_protests))
print("Размер df_events:", len(df_events))

combined = pd.concat([df_protests[features], df_events[features]], axis=0)
variances = combined.var()
threshold = 1e-8
degenerate = variances[variances <= threshold].index.tolist()

if degenerate:
    print(f"Вырожденные признаки (дисперсия <= {threshold}): {degenerate}")
    features = [f for f in features if f not in degenerate]
    print(f"Оставшиеся признаки: {features}")
else:
    print("Нет вырожденных признаков.")

combined = pd.concat([df_protests[features], df_events[features]], axis=0)
cov_matrix = np.cov(combined.T)
cov_inv = np.linalg.inv(cov_matrix)

complete_event_oktmos = []
for oktmo in df_events['oktmo'].unique():
    if check_years_completeness(oktmo, df_events_full, required_years):
        complete_event_oktmos.append(oktmo)

df_events_complete = df_events[df_events['oktmo'].isin(complete_event_oktmos)].reset_index(drop=True)

print(f"После фильтрации по годам осталось {len(df_events_complete)} муниципалитетов событий")

n_protests = df_protests.shape[0]
n_events = df_events_complete.shape[0]
distance_matrix = np.zeros((n_protests, n_events))

for i in range(n_protests):
    x = df_protests.loc[i, features].values
    for j in range(n_events):
        y = df_events_complete.loc[j, features].values
        distance_matrix[i, j] = mahalanobis(x, y, cov_inv)

row_ind, col_ind = linear_sum_assignment(distance_matrix)

df_matched_protests = df_protests.iloc[row_ind].reset_index(drop=True)
df_matched_events = df_events_complete.iloc[col_ind].reset_index(drop=True)

df_matches = df_matched_protests.join(df_matched_events, lsuffix="_protest", rsuffix="_event")
df_matches["mahalanobis_distance"] = distance_matrix[row_ind, col_ind]

df_matches.to_csv('прототип_данных_2012t.csv', index=False)
df_matches

Размер df_protests: 143
Размер df_events: 27180
Нет вырожденных признаков.
После фильтрации по годам осталось 8569 муниципалитетов событий


,region_name_protest,municipality_protest,oktmo_protest,fprotets_year,Жилье (1000 м2)_protest,Дороги (%)_protest,Канализация (1 м)_protest,Водопровод (1 метр)_protest,region_name_event,municipality_event,oktmo_event,Жилье (1000 м2)_event,Дороги (%)_event,Канализация (1 м)_event,Водопровод (1 метр)_event,mahalanobis_distance
0,Карачаево-Черкесская,Абазинский,91601000,2013.0,0.0,40.14,0.0,0.0,Смоленская,Глинковский,66609000,2.2,40.06,0.0,0.0,0.004248
1,Татарстан,Агрызский,92601000,2012.0,29.0,84.09,0.0,0.0,Курганская,Мокроусовский,37624000,13.4,84.39,0.0,0.0,0.018628
2,Удмуртская,Алнашский,94602000,2012.0,12.3,1.57,182.0,1781.0,Красноярский,Рыбинский,4647000,82.1,0.86,200.0,1765.0,0.067313
3,Башкортостан,Альшеевский,80602000,2012.0,0.0,90.00,100.0,1500.0,Свердловская,Североуральский,65755000,22.8,89.98,28.0,1409.3,0.101398
4,Тверская,Андреапольский,28602000,2011.0,20.4,0.00,0.0,400.0,Марий,Козьмодемьянск,88715000,23.0,0.00,0.0,400.0,0.001898
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138,Сахалинская,Южно-Сахалинск,64701000,2011.0,123.6,96.48,200.0,700.0,Оренбургская,Медногорск,53715000,13.6,97.00,100.0,400.0,0.169478
139,Саха,Якутск,98701000,2011.0,333.7,10.00,200.0,4500.0,Ярославская,Рыбинский,78640000,42.2,9.52,210.0,4837.0,0.231636
140,Башкортостан,Янаульский,80659000,2012.0,20.0,14.20,0.0,5800.0,Кемеровская,Яшкинский,32646000,29.7,14.90,0.0,5630.0,0.059528
141,Ярославская,Ярославль,78701000,2012.0,283.1,25.42,3200.0,12560.0,Кировская,Киров,33701000,96.6,15.20,3216.0,12420.0,0.518689


In [7]:
df_2012 = pd.read_csv('прототип_данных_2012t.csv')
df_2011 = pd.read_csv('без_протестов2011_2012t.csv')

df_2012 = df_2012[df_2012["mahalanobis_distance"] <= 10].reset_index(drop=True)

merged_df = pd.merge(
    df_2012,
    df_2011,
    left_on=['municipality_protest', 'oktmo_protest'],
    right_on=['municipality', 'oktmo'],
    how='inner'
)

from2012 = merged_df[[
    'municipality_protest', 'oktmo_protest',
    'Жилье (1000 м2)_protest', 'Дороги (%)_protest',
    'Канализация (1 м)_protest', 'Водопровод (1 метр)_protest',
    'municipality_event', 'oktmo_event',
    'Жилье (1000 м2)_event', 'Дороги (%)_event',
    'Канализация (1 м)_event', 'Водопровод (1 метр)_event', 'fprotets_year'
]]


from2012.to_csv('связка_махаланобис_2012t.csv', index=False)
from2012

,municipality_protest,oktmo_protest,Жилье (1000 м2)_protest,Дороги (%)_protest,Канализация (1 м)_protest,Водопровод (1 метр)_protest,municipality_event,oktmo_event,Жилье (1000 м2)_event,Дороги (%)_event,Канализация (1 м)_event,Водопровод (1 метр)_event,fprotets_year
0,Абазинский,91601000,0.0,40.14,0.0,0.0,Глинковский,66609000,2.2,40.06,0.0,0.0,2013.0
1,Алнашский,94602000,12.3,1.57,182.0,1781.0,Рыбинский,4647000,82.1,0.86,200.0,1765.0,2012.0
2,Апатиты,47705000,0.0,49.90,0.0,525.0,Шолоховский,60659000,4.6,50.20,0.0,500.0,2013.0
3,Архангельский,80603000,7.4,4.50,0.0,0.0,Верхоянский,98616000,20.7,4.42,0.0,0.0,2013.0
4,Асекеевский,53607000,6.2,0.00,0.0,4727.0,Неверкинский,56649000,6.8,0.00,0.0,4700.0,2012.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,Элиста,85701000,38.3,51.14,44.0,4377.0,Беловский,38602000,5.0,51.15,0.0,4400.0,2012.0
88,Южно-Сахалинск,64701000,123.6,96.48,200.0,700.0,Медногорск,53715000,13.6,97.00,100.0,400.0,2011.0
89,Якутск,98701000,333.7,10.00,200.0,4500.0,Рыбинский,78640000,42.2,9.52,210.0,4837.0,2011.0
90,Янаульский,80659000,20.0,14.20,0.0,5800.0,Яшкинский,32646000,29.7,14.90,0.0,5630.0,2012.0


Создание регрессии

In [8]:
df = pd.read_csv('общие_данные_2012t.csv')

df = df[['year', 'municipality', 'oktmo', 'Жилье (1000 м2)', 'Дороги (%)', 'Канализация (1 м)', 'Водопровод (1 метр)', 
         'region_name',
        ]]


df = df.drop_duplicates(subset=['year', 'municipality', 'oktmo'])
df = df[df['year'].isin([2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020])]



events = df
from2012 = pd.read_csv('связка_махаланобис_2012t.csv')
final = pd.DataFrame()

for _, link_row in from2012.iterrows():
    
    protest_filter = (
        (events['municipality'] == link_row['municipality_protest']) &
        (events['oktmo'] == link_row['oktmo_protest'])
    )
    protest_data = events[protest_filter].add_suffix('_protest')
    
    event_filter = (
        (events['municipality'] == link_row['municipality_event']) &
        (events['oktmo'] == link_row['oktmo_event'])
    )
    event_data = events[event_filter].add_suffix('_event')
    
    merged = protest_data.merge(
        event_data,
        left_on='year_protest',
        right_on='year_event',
        how='left'
    ).rename(columns={'year_protest': 'year'})
    

    merged = merged.drop('year_event', axis=1)

    merged['fprotets_year'] = link_row.get('fprotets_year', pd.NA)
    merged.loc[~pd.to_numeric(merged['year'], errors='coerce').astype('Int64').isin([2011, 2012, 2013]),
               'fprotets_year'] = pd.NA
    final = pd.concat([final, merged], ignore_index=True)


final.to_csv('2012t.csv', index=False)
final


,year,municipality_protest,oktmo_protest,Жилье (1000 м2)_protest,Дороги (%)_protest,Канализация (1 м)_protest,Водопровод (1 метр)_protest,region_name_protest,municipality_event,oktmo_event,Жилье (1000 м2)_event,Дороги (%)_event,Канализация (1 м)_event,Водопровод (1 метр)_event,region_name_event,fprotets_year
0,2008.0,Абазинский,91601000,0.0,0.00,0.0,2600.0,Карачаево-Черкесская,Глинковский,66609000.0,6.5,0.00,0.0,1300.0,Смоленская,NaN
1,2009.0,Абазинский,91601000,0.0,0.00,0.0,2177.0,Карачаево-Черкесская,Глинковский,66609000.0,18.7,0.00,0.0,4060.0,Смоленская,NaN
2,2010.0,Абазинский,91601000,0.0,0.00,0.0,3800.0,Карачаево-Черкесская,Глинковский,66609000.0,18.4,0.00,0.0,0.0,Смоленская,NaN
3,2011.0,Абазинский,91601000,0.0,90.40,0.0,0.0,Карачаево-Черкесская,Глинковский,66609000.0,18.4,0.00,0.0,1800.0,Смоленская,2013.0
4,2012.0,Абазинский,91601000,0.0,87.90,0.0,0.0,Карачаево-Черкесская,Глинковский,66609000.0,16.0,58.96,0.0,1400.0,Смоленская,2013.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1148,2016.0,Ярославский,78650000,0.0,31.53,180.0,4336.0,Ярославская,Уржумский,33641000.0,0.0,23.02,0.0,14300.0,Кировская,NaN
1149,2017.0,Ярославский,78650000,0.0,32.16,360.0,4350.0,Ярославская,Уржумский,33641000.0,0.0,28.20,0.0,7460.0,Кировская,NaN
1150,2018.0,Ярославский,78650000,0.0,30.91,907.0,4068.0,Ярославская,Уржумский,33641000.0,0.0,28.85,0.0,16400.0,Кировская,NaN
1151,2019.0,Ярославский,78650000,0.0,38.30,62.0,1892.0,Ярославская,Уржумский,33641000.0,0.0,29.71,0.0,6100.0,Кировская,NaN


In [9]:
def check_years_completeness(oktmo, df_events_full, required_years):
    municipality_data = df_events_full[df_events_full['oktmo'] == oktmo]
    existing_years = set(municipality_data['year'].unique())
    return existing_years.issuperset(required_years)

def matching_with_year_completeness(df_protests, df_events, df_events_full, features, required_years, cov_inv):
    matches_list = []
    
    for region in df_protests['region_name'].unique():
        sub_protests = df_protests[df_protests['region_name'] == region].reset_index(drop=True)
        sub_events = df_events[df_events['region_name'] == region].reset_index(drop=True)
        
        complete_event_munis = []
        for _, row in sub_events.iterrows():
            if check_years_completeness(row['oktmo'], df_events_full, required_years):
                complete_event_munis.append(row['oktmo'])
        
        sub_events_complete = sub_events[sub_events['oktmo'].isin(complete_event_munis)].reset_index(drop=True)
        
        if len(sub_events_complete) == 0:
            print(f"В регионе {region} нет муниципалитетов событий с полными данными за все годы")
            continue
        
        n_p = sub_protests.shape[0]
        n_e = sub_events_complete.shape[0]
        dist_mat = np.zeros((n_p, n_e))
        
        for i in range(n_p):
            x = sub_protests.loc[i, features].values
            for j in range(n_e):
                y = sub_events_complete.loc[j, features].values
                dist_mat[i, j] = mahalanobis(x, y, cov_inv)
        
        row_ind, col_ind = linear_sum_assignment(dist_mat)
        
        matched_p = sub_protests.iloc[row_ind].reset_index(drop=True)
        matched_e = sub_events_complete.iloc[col_ind].reset_index(drop=True)
        matched = matched_p.join(
            matched_e,
            lsuffix="_protest",
            rsuffix="_event"
        )
        matched["mahalanobis_distance"] = dist_mat[row_ind, col_ind]
        matches_list.append(matched)
    
    return matches_list

df_protests = pd.read_csv("протесты2012_2012t.csv")
df_protests = df_protests.drop_duplicates(subset=['region_name', 'municipality'])
df_events = pd.read_csv("единицы_без_протестов_2012t.csv")
df_events_full = pd.read_csv("общие_данные_2012t.csv")

required_years = {2010.0, 2014.0, 2015.0, 2016.0}


df_events = df_events[df_events['year'].isin([2011, 2012, 2013])].reset_index(drop=True)

features = [
    "Жилье (1000 м2)", "Дороги (%)", "Канализация (1 м)",
    "Водопровод (1 метр)"
]

df_protests = df_protests[
    ['region_name', 'municipality', 'oktmo', 'fprotets_year'] + features
].reset_index(drop=True)
df_events = df_events[
    ['region_name', 'municipality', 'oktmo'] + features
].reset_index(drop=True)

combined_all = pd.concat([df_protests[features], df_events[features]], axis=0)
variances = combined_all.var()
threshold = 1e-8
degenerate = variances[variances <= threshold].index.tolist()
if degenerate:
    print(f"Удаляем вырожденные признаки: {degenerate}")
    features = [f for f in features if f not in degenerate]
else:
    print("Вырожденных признаков нет.")

combined_all = pd.concat([df_protests[features], df_events[features]], axis=0)
cov_matrix = np.cov(combined_all.T)
cov_inv = np.linalg.inv(cov_matrix)

matches_list = matching_with_year_completeness(
    df_protests, df_events, df_events_full, features, required_years, cov_inv
)


df_matches = pd.concat(matches_list, axis=0).reset_index(drop=True)
df_matches.to_csv('прототип_данных_2012tM.csv', index=False)
df_matches

Вырожденных признаков нет.
В регионе Чеченская нет муниципалитетов событий с полными данными за все годы
В регионе Магаданская нет муниципалитетов событий с полными данными за все годы


,region_name_protest,municipality_protest,oktmo_protest,fprotets_year,Жилье (1000 м2)_protest,Дороги (%)_protest,Канализация (1 м)_protest,Водопровод (1 метр)_protest,region_name_event,municipality_event,oktmo_event,Жилье (1000 м2)_event,Дороги (%)_event,Канализация (1 м)_event,Водопровод (1 метр)_event,mahalanobis_distance
0,Карачаево-Черкесская,Абазинский,91601000,2013.0,0.0,40.14,0.0,0.0,Карачаево-Черкесская,Карачаевский,91615000,10.5,41.20,0.0,0.0,0.052705
1,Татарстан,Агрызский,92601000,2012.0,29.0,84.09,0.0,0.0,Татарстан,Бугульминский,92617000,63.1,84.69,0.0,0.0,0.038632
2,Татарстан,Казань,92701000,2011.0,413.4,0.00,600.0,8300.0,Татарстан,Алексеевское,92606151,0.0,0.00,600.0,3200.0,1.337696
3,Удмуртская,Алнашский,94602000,2012.0,12.3,1.57,182.0,1781.0,Удмуртская,Сюмсинский,94641000,14.5,0.00,0.0,1160.0,0.273671
4,Удмуртская,Завьяловский,94616000,2012.0,15.3,33.92,50.0,3260.0,Удмуртская,Малопургинский,94633000,27.7,36.07,0.0,2025.0,0.338914
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134,Ульяновская,Ульяновский,73652000,2011.0,0.0,0.00,0.0,0.0,Ульяновская,Базарносызганский,73602000,0.0,0.00,0.0,0.0,0.000000
135,Чувашская,Чебоксарский,97644000,2012.0,11.2,41.61,0.0,5200.0,Чувашская,Ядринский,97653000,4.0,40.76,100.0,3800.0,0.413798
136,Ивановская,Шуйский,24633000,2012.0,11.0,43.58,90.0,650.0,Ивановская,Кинешма,24705000,121.6,43.74,0.0,650.0,0.152568
137,Калмыкия,Элиста,85701000,2012.0,38.3,51.14,44.0,4377.0,Калмыкия,Городовиковский,85605000,4.6,57.23,0.0,300.0,1.112236


In [10]:
df_2012 = pd.read_csv('прототип_данных_2012tM.csv')
df_2011 = pd.read_csv('без_протестов2011_2012t.csv')

df_2012 = df_2012[df_2012["mahalanobis_distance"] <= 10].reset_index(drop=True)

merged_df = pd.merge(
    df_2012,
    df_2011,
    left_on=['municipality_protest', 'oktmo_protest'],
    right_on=['municipality', 'oktmo'],
    how='inner'
)

from2012 = merged_df[[
    'municipality_protest', 'oktmo_protest',
    'Жилье (1000 м2)_protest', 'Дороги (%)_protest',
    'Канализация (1 м)_protest', 'Водопровод (1 метр)_protest',
    'municipality_event', 'oktmo_event',
    'Жилье (1000 м2)_event', 'Дороги (%)_event',
    'Канализация (1 м)_event', 'Водопровод (1 метр)_event', 'fprotets_year'
]]


from2012.to_csv('связка_махаланобис_2012tM.csv', index=False)
from2012


,municipality_protest,oktmo_protest,Жилье (1000 м2)_protest,Дороги (%)_protest,Канализация (1 м)_protest,Водопровод (1 метр)_protest,municipality_event,oktmo_event,Жилье (1000 м2)_event,Дороги (%)_event,Канализация (1 м)_event,Водопровод (1 метр)_event,fprotets_year
0,Абазинский,91601000,0.0,40.14,0.0,0.0,Карачаевский,91615000,10.5,41.20,0.0,0.0,2013.0
1,Алнашский,94602000,12.3,1.57,182.0,1781.0,Сюмсинский,94641000,14.5,0.00,0.0,1160.0,2012.0
2,Архангельский,80603000,7.4,4.50,0.0,0.0,Бураевский,80618000,15.2,6.40,0.0,200.0,2013.0
3,Аургазинский,80605000,2.1,12.00,0.0,30100.0,Ермекеевский,80625000,2.9,67.80,0.0,25200.0,2011.0
4,Янаульский,80659000,20.0,14.20,0.0,5800.0,Бирский,80613000,25.0,17.00,0.0,6200.0,2012.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,Ульяновский,73652000,0.0,0.00,0.0,0.0,Базарносызганский,73602000,0.0,0.00,0.0,0.0,2011.0
85,Чебоксарский,97644000,11.2,41.61,0.0,5200.0,Ядринский,97653000,4.0,40.76,100.0,3800.0,2012.0
86,Шуйский,24633000,11.0,43.58,90.0,650.0,Кинешма,24705000,121.6,43.74,0.0,650.0,2012.0
87,Элиста,85701000,38.3,51.14,44.0,4377.0,Городовиковский,85605000,4.6,57.23,0.0,300.0,2012.0


In [11]:
df = pd.read_csv('общие_данные_2012t.csv')

df = df[['year', 'municipality', 'oktmo', 'Жилье (1000 м2)', 'Дороги (%)', 'Канализация (1 м)', 'Водопровод (1 метр)', 
         'region_name',
        ]]


df = df.drop_duplicates(subset=['year', 'municipality', 'oktmo'])
df = df[df['year'].isin([2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020])]



events = df
from2012 = pd.read_csv('связка_махаланобис_2012tM.csv')
final = pd.DataFrame()

for _, link_row in from2012.iterrows():
    
    protest_filter = (
        (events['municipality'] == link_row['municipality_protest']) &
        (events['oktmo'] == link_row['oktmo_protest'])
    )
    protest_data = events[protest_filter].add_suffix('_protest')
    
    event_filter = (
        (events['municipality'] == link_row['municipality_event']) &
        (events['oktmo'] == link_row['oktmo_event'])
    )
    event_data = events[event_filter].add_suffix('_event')
    
    merged = protest_data.merge(
        event_data,
        left_on='year_protest',
        right_on='year_event',
        how='left'
    ).rename(columns={'year_protest': 'year'})
    

    merged = merged.drop('year_event', axis=1)
    merged['fprotets_year'] = link_row.get('fprotets_year', pd.NA)
    merged.loc[~pd.to_numeric(merged['year'], errors='coerce').astype('Int64').isin([2011, 2012, 2013]),
               'fprotets_year'] = pd.NA
    final = pd.concat([final, merged], ignore_index=True)


final.to_csv('2012tM.csv', index=False)
final


,year,municipality_protest,oktmo_protest,Жилье (1000 м2)_protest,Дороги (%)_protest,Канализация (1 м)_protest,Водопровод (1 метр)_protest,region_name_protest,municipality_event,oktmo_event,Жилье (1000 м2)_event,Дороги (%)_event,Канализация (1 м)_event,Водопровод (1 метр)_event,region_name_event,fprotets_year
0,2008.0,Абазинский,91601000,0.0,0.00,0.0,2600.0,Карачаево-Черкесская,Карачаевский,91615000.0,16.3,0.00,0.0,3200.0,Карачаево-Черкесская,NaN
1,2009.0,Абазинский,91601000,0.0,0.00,0.0,2177.0,Карачаево-Черкесская,Карачаевский,91615000.0,12.7,0.00,120.0,13350.0,Карачаево-Черкесская,NaN
2,2010.0,Абазинский,91601000,0.0,0.00,0.0,3800.0,Карачаево-Черкесская,Карачаевский,91615000.0,12.7,0.00,0.0,1000.0,Карачаево-Черкесская,NaN
3,2011.0,Абазинский,91601000,0.0,90.40,0.0,0.0,Карачаево-Черкесская,Карачаевский,91615000.0,10.5,41.20,0.0,0.0,Карачаево-Черкесская,2013.0
4,2012.0,Абазинский,91601000,0.0,87.90,0.0,0.0,Карачаево-Черкесская,Карачаевский,91615000.0,10.5,0.77,0.0,0.0,Карачаево-Черкесская,2013.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1114,2016.0,Якутск,98701000,0.0,25.00,1900.0,1600.0,Саха,Нерюнгри,98660101.0,0.0,0.00,0.0,100.0,Саха,NaN
1115,2017.0,Якутск,98701000,0.0,25.00,0.0,1100.0,Саха,Нерюнгри,98660101.0,0.0,0.00,0.0,3580.0,Саха,NaN
1116,2018.0,Якутск,98701000,0.0,25.00,0.0,3510.0,Саха,Нерюнгри,98660101.0,0.0,0.00,200.0,0.0,Саха,NaN
1117,2019.0,Якутск,98701000,0.0,29.05,1320.0,2830.0,Саха,Нерюнгри,98660101.0,0.0,0.00,0.0,2460.0,Саха,NaN


In [12]:
def check_years_completeness(oktmo, df_events_full, required_years):
    municipality_data = df_events_full[df_events_full['oktmo'] == oktmo]
    existing_years = set(municipality_data['year'].unique())
    return existing_years.issuperset(required_years)


df_protests = pd.read_csv("протесты2012_2012t.csv")
df_protests = df_protests.drop_duplicates(subset=['region_name', 'municipality'])
df_events = pd.read_csv("единицы_без_протестов_2012t.csv")
df_events_full = pd.read_csv("общие_данные_2012t.csv")

required_years = {2010.0, 2014.0, 2015.0, 2016.0}


df_events = df_events[df_events['year'].isin([2011, 2012, 2013])].reset_index(drop=True)

features = [
    "Жилье (1000 м2)", "Дороги (%)", "Канализация (1 м)",
    "Водопровод (1 метр)"
]

df_protests = df_protests[['region_name', 'municipality', 'oktmo', 'fprotets_year'] + features].reset_index(drop=True)
df_events = df_events[['region_name', 'municipality', 'oktmo'] + features].reset_index(drop=True)

exclude_dict = {
    'Абакан': ['Усть-Абаканский', 'Алтайский', 'Минусинский', 'Минусинск', 'Черногорск'],
    'Анапа': ['Темрюкский', 'Крымский', 'Новороссийск'],
    'Архангельск': ['Северодвинск', 'Приморский'],
    'Архаринский': ['Облученский', 'Верхнебуреинский', 'Бурейский'],
    'Бабаюрт': ['Шелковской', 'Хасавюртовский', 'Кизилюртовский', 'Кумторкалинский', 'Кизлярский'],
    'Балашиха': ['Королев', 'Люберцы', 'Богородский', 'Щелково', 'Раменский'],
    'Благовещенск': ['Благовещенский', 'Кушнаревский', 'Имангуловский'],
    'Бокситогорский': ['Тихвинский', 'Любытинский', 'Хвойнинский', 'Чагодощенский', 'Бабаевский'],
    'Брянск': ['Брянский', 'Сельцо', 'Фокино', 'Титовское'],
    'Варгашинский': ['Кетовский', 'Курган', 'Половинский', 'Лебяжьевский', 'Мокроусовский', 'Упоровский', 'Белозерский'],
    'Вологда': ['Вологодский'],
    'Воронеж': ['Хохольский', 'Семилукский', 'Рамонский', 'Новоусманский', 'Каширский'],
    'Гатчинский': ['Лужский', 'Волосовский', 'Тосненский', 'Пушкин', 'Ломоносовский'],
    'Геленджик': ['Туапсинский', 'Северский', 'Абинский', 'Крымский', 'Новороссийск'],
    'Горно-Алтайск': ['Майминский'],
    'Грязовецкий': ['Вологодский', 'Междуреченский', 'Солигаличский', 'Буйский', 'Любимский', 'Первомайский', 'Пошехонский'],
    'Гулькевичский': ['Кавказский', 'Новоалександровский', 'Новокубанский', 'Курганинский', 'Тбилисский'],
    'Гурьевский': ['Зеленоградский', 'Калининград', 'Полесский', 'Гвардейский', 'Багратионовский', 'Ладушкинский'],
    'Добрянский': ['Краснокамский', 'Ильинский', 'Юсьвинский', 'Березники', 'Александровский', 'Губаха', 'Чусовской', 'Пермский'],
    'Домодедово': ['Подольск', 'Ленинский', 'Раменский', 'Чехов', 'Ступино', 'Щербинка'],
    'Дубна': ['Кимрский', 'Кимры', 'Талдомский', 'Дмитровский', 'Конаковский'],
    'Завьяловский': ['Ижевск', 'Увинский', 'Малопургинский', 'Сарапульский', 'Чайковский', 'Воткинский', 'Якшур-Бодьинский'],
    'Ивановский': ['Иваново', 'Фурмановский', 'Комсомольский', 'Тейковский', 'Лежневский', 'Шуйский', 'Родниковский'],
    'Иркутск': ['Иркутский', 'Ангарское', 'Ангарск', 'Шелеховский'],
    'Казань': ['Лаишевский', 'Пестречинский', 'Высокогорский', 'Зеленодольский', 'Верхнеуслонский', 'Новоильмовское'],
    'Калуга': ['Перемышльский', 'Ферзиковский', 'Малоярославецкий', 'Дзержинский', 'Бабынинский'],
    'Кингисеппский': ['Сланцевский', 'Волосовский', 'Ломоносовский', 'Сосновоборский'],
    'Кинельский': ['Кинель', 'Волжский', 'Красноярский', 'Кинель-Черкасский', 'Богатовский', 'Нефтегорский', 'Приволжье'],
    'Кола': ['Североморск', 'Александровск', 'Кольский', 'Кильдинстрой'],
    'Комсомольск-на-Амуре': ['Комсомольский'],
    'Краснодар': ['Динский', 'Красноармейский', 'Тахтамукайский'],
    'Красноярский': ['Володарский', 'Приволжский', 'Наримановский', 'Харабалинский'],
    'Курганинский': ['Лабинский', 'Новокубанский', 'Гулькевичский', 'Тбилисский', 'Шовгеновский', 'Кошехабльский'],
    'Макаровский': ['Долинский', 'Томаринский', 'Углегорский', 'Поронайский', 'Вахрушев'],
    'Мезенский': ['Заполярный', 'Усть-Цилемский', 'Лешуконский', 'Пинежский', 'Приморский'],
    'Михайловский': ['Новомосковск', 'Кимовский', 'Скопинский', 'Пронский', 'Захаровский', 'Серебряные Пруды'],
    'Новороссийск': ['Новороссийский'],
    'Одинцовский': ['Власиха', 'Внуковское', 'Кокошкинское', 'Можайский', 'Красногорск', 'Марушкинское', 'Наро-Фоминский', 'Рузский', 'Истра'],
    'Омск': ['Омский'],
    'Оренбург': ['Оренбургский', 'Сакмарский'],
    'Орёл': ['Орловский'],
    'Петропавловск-Камчатский': ['Вилючинский', 'Елизовский'],
    'Пинежский': ['Лешуконский', 'Мезенский', 'Приморский', 'Холмогорский', 'Виноградовский', 'Верхнетоемский', 'Удорский'],
    'Почепский': ['Трубчевский', 'Погарский', 'Унечский', 'Мглинский', 'Клетнянский', 'Жирятинский', 'Выгоничский', 'Титовское'],
    'Пушкинский': ['Мытищи', 'Дмитровский', 'Сергиево-Посадский', 'Красноармейск', 'Щёлково', 'Восточный', 'Ивантеевка'],
    'Ростов': ['Ростовский'],
    'Саратовский': ['Красноармейский', 'Саратов', 'Лысогорский', 'Татищевский'],
    'Северодвинск': ['Приморский', 'Архангельск'],
    'Северский': ['Туапсинский', 'Геленджик', 'Абинский', 'Горячий Ключ', 'Тахтамукайский', 'Красноармейский'],
    'Тамбов': ['Тамбовский', 'Котовск'],
    'Таштыпский': ['Абаза', 'Таштагольский', 'Турочакский', 'Улаганский', 'Бай-Тайгинский', 'Барун-Хемчикский', 'Сут-Хольский', 'Шушенский', 'Бейский', 'Аскизский'],
    'Тверь': ['Калининский'],
    'Тимашевский': ['Кореновский', 'Динский', 'Калининский', 'Приморско-Ахтарский', 'Брюховецкий'],
    'Тольятти': ['Жигулевск', 'Ставропольский'],
    'Тотемский': ['Бабушкинский', 'Чухломский', 'Солигаличский', 'Междуреченский', 'Сокольский', 'Сямженский', 'Верховажский', 'Тарногский', 'Нюксенский'],
    'Тула': ['Щекинский', 'Дубенский', 'Алексин', 'Ясногорский', 'Веневский', 'Киреевский', 'Ленинский'],
    'Усть-Джегутинский': ['Усть-Джегутинское'],
    'Учалинский': ['Белорецкий', 'Катав-Ивановский', 'Верхнеуральский', 'Уйский', 'Чебаркульский', 'Миасский', 'Златоустовский'],
    'Хасавюрт': ['Хасавюртовский', 'Новолакский', 'Ахвахский'],
    'Целинский': ['Егорлыкский', 'Зерноградский', 'Сальский', 'Песчанокопский', 'Белоглинский'],
    'Чернский': ['Мценский', 'Корсаковский', 'Каменский', 'Тепло-Огаревский', 'Плавский', 'Арсеньевский', 'Славный'],
    'Ярославль': ['Ярославский'],
     'Советский': ['Будённовский', 'Степновский', 'Кировский', 'Георгиевский', 'Новоселицкий'], 
     'Ульяновский': ['Ульяновск', 'Чердаклинский', 'Новоульяновск', 'Сенгилеевский', 'Теренгульский', 'Майнский', 'Цильнинский', 'Тетюшский'],
     "Владивостокский": ["Хасанский", "Надеждинский", "Артемовский", "Шкотовский", "Большой Камень", "Фокино"],
    "Воротынский": ["Пильнинский", "Спасский", "Лысковский", "Воскресенский", "Юринский", "Горномарийский", "Ядринский"],
    "Джейрах": ["Итум-Калинский", "Ачхой-Мартановский", "Сунженский", "Пригородный", "Владикавказ"],
    "Кизилюртовский": ["Кумторкалинский", "Хунзахский", "Буйнакский", "Казбековский", "Хасавюртовский",
                       "Кизилюрт", "Цумадинский", "Бабаюртовский", "Ахвахский"],
    "Кичменгско-Городецкий": ["Нюксенский", "Бабушкинский", "Никольский", "Вохомский", "Опаринский", "Подосиновский", "Великоустюгский"],
    "Красногвардейский": ["Усть-Лабинский", "Шовгеновский", "Белореченский", "Теучежский", "Краснодар", "Динский"],
    "Красноярск": ["Березовский", "Дивногорск", "Емельяновский"],
    "Нестеровский": ["Краснознаменский", "Гусевский", "Озерский"],
    "Сегежский": ["Беломорский", "Муезерский", "Медвежьегорский", "Онежский"],
    "Ульяновский": ["Ульяновск", "Цильнинский", "Майнский", "Тереньгульский",
                    "Сенгилеевский", "Новоульяновск", "Чердаклинский", "Тетюшский"],
    "Хасавюртовский": ["Хасавюрт", "Новолакский", "Ахвахский", "Гудермесский", "Шелковский",
                       "Бабаюртовский", "Хунзахский", "Кизилюртовский", "Кизилюрт", "Казбековский"],
    "Чайковский": ["Еловский", "Куединский", "Янаульский", "Нефтекамск", "Камбарский",
                   "Сарапульский", "Завьяловский", "Воткинский"],
    "Шовгеновский": ["Курганинский", "Кошехабльский", "Гиагинский", "Белореченский", "Красногвардейский", "Усть-Лабинский"],
    "Южно-Сахалинск": ["Корсаковский", "Анивский", "Холмский", "Долинский"]
}


excluded_pairs = set()
for protest, events in exclude_dict.items():
    for event in events:
        excluded_pairs.add((protest, event))

combined_all = pd.concat([df_protests[features], df_events[features]], axis=0)
variances = combined_all.var()
threshold = 1e-8
degenerate = variances[variances <= threshold].index.tolist()
if degenerate:
    print(f"Удаляем вырожденные признаки: {degenerate}")
    features = [f for f in features if f not in degenerate]
else:
    print("Вырожденных признаков нет.")

combined_all = pd.concat([df_protests[features], df_events[features]], axis=0)
cov_matrix = np.cov(combined_all.T)
cov_inv = np.linalg.inv(cov_matrix)

# Matching внутри каждого региона с проверкой ограничений
matches_list = []

for region in df_protests['region_name'].unique():
    sub_protests = df_protests[df_protests['region_name'] == region].reset_index(drop=True)
    sub_events = df_events[df_events['region_name'] == region].reset_index(drop=True)
    
    # Фильтрация событий: полнота данных + проверка на исключения
    valid_events = []
    for _, event_row in sub_events.iterrows():
        event_name = event_row['municipality']
        oktmo = event_row['oktmo']
        
        # Проверка полноты данных
        has_full_data = check_years_completeness(oktmo, df_events_full, required_years)
        
        # Проверка, что нет ни одного протеста, для которого это событие запрещено
        is_allowed = all(
            (protest_name, event_name) not in excluded_pairs
            for protest_name in sub_protests['municipality']
        )
        
        if has_full_data and is_allowed:
            valid_events.append(event_row)
    
    if not valid_events:
        print(f"В регионе {region} нет подходящих муниципалитетов событий")
        continue
    
    sub_events_valid = pd.DataFrame(valid_events).reset_index(drop=True)
    
    n_p = sub_protests.shape[0]
    n_e = sub_events_valid.shape[0]
    
    if n_e == 0:
        continue
    
    dist_mat = np.zeros((n_p, n_e))
    
    # Заполнение матрицы расстояний
    for i in range(n_p):
        x = sub_protests.loc[i, features].values
        for j in range(n_e):
            y = sub_events_valid.loc[j, features].values
            dist_mat[i, j] = mahalanobis(x, y, cov_inv)
    
    # Венгерский алгоритм
    row_ind, col_ind = linear_sum_assignment(dist_mat)
    
    # Сбор результатов
    matched_p = sub_protests.iloc[row_ind].reset_index(drop=True)
    matched_e = sub_events_valid.iloc[col_ind].reset_index(drop=True)
    
    matched = matched_p.join(
        matched_e,
        lsuffix="_protest",
        rsuffix="_event"
    )
    matched["mahalanobis_distance"] = dist_mat[row_ind, col_ind]
    matches_list.append(matched)




df_matches = pd.concat(matches_list, axis=0).reset_index(drop=True)
df_matches.to_csv('прототип_данных_2012tM2.csv', index=False)
df_matches

Вырожденных признаков нет.
В регионе Чеченская нет подходящих муниципалитетов событий
В регионе Магаданская нет подходящих муниципалитетов событий


,region_name_protest,municipality_protest,oktmo_protest,fprotets_year,Жилье (1000 м2)_protest,Дороги (%)_protest,Канализация (1 м)_protest,Водопровод (1 метр)_protest,region_name_event,municipality_event,oktmo_event,Жилье (1000 м2)_event,Дороги (%)_event,Канализация (1 м)_event,Водопровод (1 метр)_event,mahalanobis_distance
0,Карачаево-Черкесская,Абазинский,91601000,2013.0,0.0,40.14,0.0,0.0,Карачаево-Черкесская,Карачаевский,91615000,10.5,41.20,0.0,0.0,0.052705
1,Татарстан,Агрызский,92601000,2012.0,29.0,84.09,0.0,0.0,Татарстан,Бугульминский,92617000,63.1,84.69,0.0,0.0,0.038632
2,Татарстан,Казань,92701000,2011.0,413.4,0.00,600.0,8300.0,Татарстан,Алексеевское,92606151,0.0,0.00,600.0,3200.0,1.337696
3,Удмуртская,Алнашский,94602000,2012.0,12.3,1.57,182.0,1781.0,Удмуртская,Сюмсинский,94641000,14.5,0.00,0.0,1160.0,0.273671
4,Удмуртская,Завьяловский,94616000,2012.0,15.3,33.92,50.0,3260.0,Удмуртская,Селтинский,94639000,15.0,28.69,0.0,4000.0,0.355475
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134,Ульяновская,Ульяновский,73652000,2011.0,0.0,0.00,0.0,0.0,Ульяновская,Базарносызганский,73602000,0.0,0.00,0.0,0.0,0.000000
135,Чувашская,Чебоксарский,97644000,2012.0,11.2,41.61,0.0,5200.0,Чувашская,Ядринский,97653000,4.0,40.76,100.0,3800.0,0.413798
136,Ивановская,Шуйский,24633000,2012.0,11.0,43.58,90.0,650.0,Ивановская,Кинешма,24705000,121.6,43.74,0.0,650.0,0.152568
137,Калмыкия,Элиста,85701000,2012.0,38.3,51.14,44.0,4377.0,Калмыкия,Городовиковский,85605000,4.6,57.23,0.0,300.0,1.112236


In [13]:
df_2012 = pd.read_csv('прототип_данных_2012tM2.csv')
df_2011 = pd.read_csv('без_протестов2011_2012t.csv')

df_2012 = df_2012[df_2012["mahalanobis_distance"] <= 10].reset_index(drop=True)

merged_df = pd.merge(
    df_2012,
    df_2011,
    left_on=['municipality_protest', 'oktmo_protest'],
    right_on=['municipality', 'oktmo'],
    how='inner'
)

from2012 = merged_df[[
    'municipality_protest', 'oktmo_protest',
    'Жилье (1000 м2)_protest', 'Дороги (%)_protest',
    'Канализация (1 м)_protest', 'Водопровод (1 метр)_protest',
    'municipality_event', 'oktmo_event',
    'Жилье (1000 м2)_event', 'Дороги (%)_event',
    'Канализация (1 м)_event', 'Водопровод (1 метр)_event', 'fprotets_year'
]]


from2012.to_csv('связка_махаланобис_2012tM2.csv', index=False)
from2012

,municipality_protest,oktmo_protest,Жилье (1000 м2)_protest,Дороги (%)_protest,Канализация (1 м)_protest,Водопровод (1 метр)_protest,municipality_event,oktmo_event,Жилье (1000 м2)_event,Дороги (%)_event,Канализация (1 м)_event,Водопровод (1 метр)_event,fprotets_year
0,Абазинский,91601000,0.0,40.14,0.0,0.0,Карачаевский,91615000,10.5,41.20,0.0,0.0,2013.0
1,Алнашский,94602000,12.3,1.57,182.0,1781.0,Сюмсинский,94641000,14.5,0.00,0.0,1160.0,2012.0
2,Архангельский,80603000,7.4,4.50,0.0,0.0,Бураевский,80618000,15.2,6.40,0.0,200.0,2013.0
3,Аургазинский,80605000,2.1,12.00,0.0,30100.0,Ермекеевский,80625000,2.9,67.80,0.0,25200.0,2011.0
4,Янаульский,80659000,20.0,14.20,0.0,5800.0,Бирский,80613000,25.0,17.00,0.0,6200.0,2012.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,Ульяновский,73652000,0.0,0.00,0.0,0.0,Базарносызганский,73602000,0.0,0.00,0.0,0.0,2011.0
85,Чебоксарский,97644000,11.2,41.61,0.0,5200.0,Ядринский,97653000,4.0,40.76,100.0,3800.0,2012.0
86,Шуйский,24633000,11.0,43.58,90.0,650.0,Кинешма,24705000,121.6,43.74,0.0,650.0,2012.0
87,Элиста,85701000,38.3,51.14,44.0,4377.0,Городовиковский,85605000,4.6,57.23,0.0,300.0,2012.0


In [14]:
df = pd.read_csv('общие_данные_2012t.csv')

df = df[['year', 'municipality', 'oktmo', 'Жилье (1000 м2)', 'Дороги (%)', 'Канализация (1 м)', 'Водопровод (1 метр)', 
         'region_name',
        ]]


df = df.drop_duplicates(subset=['year', 'municipality', 'oktmo'])
df = df[df['year'].isin([2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020])]



events = df
from2012 = pd.read_csv('связка_махаланобис_2012tM2.csv')
final = pd.DataFrame()

for _, link_row in from2012.iterrows():
    
    protest_filter = (
        (events['municipality'] == link_row['municipality_protest']) &
        (events['oktmo'] == link_row['oktmo_protest'])
    )
    protest_data = events[protest_filter].add_suffix('_protest')
    
    event_filter = (
        (events['municipality'] == link_row['municipality_event']) &
        (events['oktmo'] == link_row['oktmo_event'])
    )
    event_data = events[event_filter].add_suffix('_event')
    
    merged = protest_data.merge(
        event_data,
        left_on='year_protest',
        right_on='year_event',
        how='left'
    ).rename(columns={'year_protest': 'year'})
    

    merged = merged.drop('year_event', axis=1)
    merged['fprotets_year'] = link_row.get('fprotets_year', pd.NA)
    merged.loc[~pd.to_numeric(merged['year'], errors='coerce').astype('Int64').isin([2011, 2012, 2013]),
               'fprotets_year'] = pd.NA
    final = pd.concat([final, merged], ignore_index=True)


final.to_csv('2012tM2.csv', index=False)
final


,year,municipality_protest,oktmo_protest,Жилье (1000 м2)_protest,Дороги (%)_protest,Канализация (1 м)_protest,Водопровод (1 метр)_protest,region_name_protest,municipality_event,oktmo_event,Жилье (1000 м2)_event,Дороги (%)_event,Канализация (1 м)_event,Водопровод (1 метр)_event,region_name_event,fprotets_year
0,2008.0,Абазинский,91601000,0.0,0.00,0.0,2600.0,Карачаево-Черкесская,Карачаевский,91615000.0,16.3,0.00,0.0,3200.0,Карачаево-Черкесская,NaN
1,2009.0,Абазинский,91601000,0.0,0.00,0.0,2177.0,Карачаево-Черкесская,Карачаевский,91615000.0,12.7,0.00,120.0,13350.0,Карачаево-Черкесская,NaN
2,2010.0,Абазинский,91601000,0.0,0.00,0.0,3800.0,Карачаево-Черкесская,Карачаевский,91615000.0,12.7,0.00,0.0,1000.0,Карачаево-Черкесская,NaN
3,2011.0,Абазинский,91601000,0.0,90.40,0.0,0.0,Карачаево-Черкесская,Карачаевский,91615000.0,10.5,41.20,0.0,0.0,Карачаево-Черкесская,2013.0
4,2012.0,Абазинский,91601000,0.0,87.90,0.0,0.0,Карачаево-Черкесская,Карачаевский,91615000.0,10.5,0.77,0.0,0.0,Карачаево-Черкесская,2013.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1114,2016.0,Якутск,98701000,0.0,25.00,1900.0,1600.0,Саха,Нерюнгри,98660101.0,0.0,0.00,0.0,100.0,Саха,NaN
1115,2017.0,Якутск,98701000,0.0,25.00,0.0,1100.0,Саха,Нерюнгри,98660101.0,0.0,0.00,0.0,3580.0,Саха,NaN
1116,2018.0,Якутск,98701000,0.0,25.00,0.0,3510.0,Саха,Нерюнгри,98660101.0,0.0,0.00,200.0,0.0,Саха,NaN
1117,2019.0,Якутск,98701000,0.0,29.05,1320.0,2830.0,Саха,Нерюнгри,98660101.0,0.0,0.00,0.0,2460.0,Саха,NaN
